# Part 6: AgentReview Demo - 5단계 Peer Review 시뮬레이션

**SNU AI Psychology Workshop - February 2026**

---

## AgentReview (EMNLP 2024)

실제 **ICLR** (International Conference on Learning Representations) peer review 프로세스를 LLM으로 시뮬레이션하는 프레임워크입니다.

**5단계 파이프라인:**
1. **Reviewer Assessment**: 3명의 리뷰어가 독립적으로 논문 평가
2. **Author-Reviewer Discussion**: 저자가 rebuttal 제출
3. **Reviewer-AC Discussion**: Area Chair가 리뷰어 간 토론 중재
4. **Meta-Review**: AC가 종합 리뷰 작성
5. **Paper Decision**: Accept/Reject 최종 결정

**참고 자료:**
- 📄 [논문](https://arxiv.org/abs/2406.12708)
- 🌐 [웹사이트](https://agentreview.github.io/)
- 💻 [GitHub](https://github.com/Ahren09/AgentReview)
- 💡 [데모](https://huggingface.co/spaces/Ahren09/AgentReview)
- 📦 [Full 데이터 (Dropbox)](https://www.dropbox.com/scl/fi/l17brtbzsy3xwflqd58ja/AgentReview_Paper_Data.zip?rlkey=vldiexmgzi7zycmz7pumgbooc&st=b6g3nkry&dl=0)

---
## Setup

In [ ]:
# Cell 1: 환경 설정
import os
import sys

try:
    from google.colab import drive
    drive.mount('/content/drive/')
    WORKSHOP_DIR = "/content/drive/MyDrive/aiworkshop_Feb2026/"
    os.makedirs(WORKSHOP_DIR, exist_ok=True)
    os.chdir(WORKSHOP_DIR)
    IN_COLAB = True
    print("🌐 Colab 환경")
except ImportError:
    # 로컬: 프로젝트 루트로 이동
    current_dir = os.getcwd()
    if current_dir.endswith('notebooks'):
        os.chdir('..')
    WORKSHOP_DIR = os.getcwd()
    IN_COLAB = False
    print("💻 로컬 환경")

print(f"작업 폴더: {WORKSHOP_DIR}")

In [ ]:
# Cell 2: 패키지 설치
if IN_COLAB:
    print("📦 설치 중...")
    %pip install openai python-dotenv llama-index-readers-file -q
    !git clone https://github.com/ahren09/agentreview.git 2>/dev/null || echo "✓ AgentReview 이미 존재"
    print("✅ 완료!")
else:
    print("✅ 로컬: pyproject.toml 사용")
    import subprocess
    import os
    
    # WORKSHOP_DIR 기준으로 확인
    agentreview_path = os.path.join(WORKSHOP_DIR, "agentreview")
    
    if not os.path.exists(agentreview_path):
        print("📥 AgentReview 클론 중...")
        subprocess.run(["git", "clone", "https://github.com/ahren09/agentreview.git", agentreview_path], capture_output=True)
        print("✅ AgentReview 클론 완료")
    else:
        print("✓ AgentReview 이미 존재")
    
    # llama-index 설치 확인
    try:
        import llama_index
        print("✓ llama-index 설치됨")
    except ImportError:
        print("📥 llama-index 설치 중...")
        subprocess.run(["pip", "install", "llama-index-readers-file"], capture_output=True)
        print("✅ llama-index 설치 완료")

In [ ]:
# Cell 3: API Key
if IN_COLAB:
    try:
        from google.colab import userdata
        OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
        print("✅ Colab Secrets에서 로딩")
    except:
        OPENAI_API_KEY = None
        print("⚠️ Colab Secrets에 OPENAI_API_KEY 추가 필요")
else:
    try:
        from dotenv import load_dotenv
        load_dotenv()
        OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
        if OPENAI_API_KEY:
            print("✅ .env 파일에서 로딩")
        else:
            print("⚠️ .env 파일에 OPENAI_API_KEY 추가 필요")
    except:
        OPENAI_API_KEY = None

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY or ''
print(f"\n🔑 OpenAI: {'설정됨' if OPENAI_API_KEY else '없음'}")

---
## 데이터 확인

In [ ]:
# Cell 4: 샘플 데이터 확인
import os
import json

data_dir = "data/ICLR2022"

if os.path.exists(data_dir):
    print("✅ ICLR2022 샘플 데이터 발견!")
    
    # 논문 ID 확인
    with open(f"{data_dir}/id2decision.json") as f:
        id2decision = json.load(f)
    
    print(f"\n📄 포함된 논문 ({len(id2decision)}개):")
    for pid, decision in id2decision.items():
        print(f"   - Paper {pid}: {decision}")
    
    USE_AGENTREVIEW = True
else:
    print(f"❌ 샘플 데이터를 찾을 수 없습니다: {os.path.abspath(data_dir)}")
    print(f"   현재 작업 폴더: {os.getcwd()}")
    print(f"\n해결 방법:")
    print(f"   1. 프로젝트 루트 폴더에서 노트북 실행")
    print(f"   2. 또는 Cell 1을 먼저 실행하여 작업 폴더 설정")
    USE_AGENTREVIEW = False

---
## 리뷰어 페르소나 설정

In [ ]:
# Cell 5: 리뷰어 설정

# 리뷰어 페르소나 속성 (조합 가능!)
# 
# 1. Intention (의도):
#    - benign: 호의적, 높은 점수 선호
#    - malicious: 악의적, 낮은 점수 선호
#    - BASELINE: 중립
#
# 2. Knowledgeability (전문성):
#    - knowledgeable: 전문가 수준
#    - unknowledgeable: 비전문가
#    - BASELINE: 보통
#
# 3. Commitment (책임감):
#    - responsible: 성실한 리뷰
#    - irresponsible: 불성실한 리뷰
#    - BASELINE: 보통
#
# 4. Fame (저자 인지도):
#    - authors_are_famous: 유명 저자로 인식
#    - authors_are_unfamous: 무명 저자로 인식
#
# 조합 예시:
#    - "knowledgeable_responsible": 전문가이면서 성실
#    - "malicious_unknowledgeable": 악의적이면서 비전문가
#    - "benign_irresponsible": 호의적이지만 불성실

# 기본 설정: 모든 리뷰어가 일반적 (BASELINE)
baseline_setting = {
    "reviewer": ["BASELINE", "BASELINE", "BASELINE"],
    "AC": ["BASELINE"],
    "author": ["BASELINE"],
    "global_settings": {
        "provides_numeric_rating": ['reviewer', 'ac'],
        "persons_aware_of_authors_identities": []
    }
}

# 실험 1: 악의적 리뷰어 1명
malicious_setting = {
    "reviewer": ["malicious", "BASELINE", "BASELINE"],
    "AC": ["BASELINE"],
    "author": ["BASELINE"],
    "global_settings": {
        "provides_numeric_rating": ['reviewer', 'ac'],
        "persons_aware_of_authors_identities": []
    }
}

# 실험 2: 호의적 리뷰어 1명
benign_setting = {
    "reviewer": ["benign", "BASELINE", "BASELINE"],
    "AC": ["BASELINE"],
    "author": ["BASELINE"],
    "global_settings": {
        "provides_numeric_rating": ['reviewer', 'ac'],
        "persons_aware_of_authors_identities": []
    }
}

# 실험 3: 전문가 vs 비전문가 혼합
expert_mix_setting = {
    "reviewer": ["knowledgeable_responsible", "unknowledgeable", "BASELINE"],
    "AC": ["BASELINE"],
    "author": ["BASELINE"],
    "global_settings": {
        "provides_numeric_rating": ['reviewer', 'ac'],
        "persons_aware_of_authors_identities": []
    }
}

# 실험 4: 최악의 시나리오 (악의적 + 비전문가 + 불성실)
worst_case_setting = {
    "reviewer": ["malicious_unknowledgeable_irresponsible", "BASELINE", "BASELINE"],
    "AC": ["BASELINE"],
    "author": ["BASELINE"],
    "global_settings": {
        "provides_numeric_rating": ['reviewer', 'ac'],
        "persons_aware_of_authors_identities": []
    }
}

# 실험 5: 유명 저자 효과 테스트
fame_bias_setting = {
    "reviewer": ["authors_are_famous", "authors_are_unfamous", "BASELINE"],
    "AC": ["BASELINE"],
    "author": ["BASELINE"],
    "global_settings": {
        "provides_numeric_rating": ['reviewer', 'ac'],
        "persons_aware_of_authors_identities": []
    }
}

print("✅ 리뷰어 페르소나 정의 완료")
print("\n사용 가능한 설정:")
print("  1. baseline_setting: 모두 중립")
print("  2. malicious_setting: 악의적 리뷰어 1명")
print("  3. benign_setting: 호의적 리뷰어 1명")
print("  4. expert_mix_setting: 전문가 vs 비전문가")
print("  5. worst_case_setting: 최악의 리뷰어")
print("  6. fame_bias_setting: 저자 유명도 편향 테스트")
print("\n💡 Cell 7에서 원하는 설정을 선택하세요!")

---
## AgentReview 실행

In [ ]:
# Cell 6: AgentReview 모듈 로딩
if USE_AGENTREVIEW and OPENAI_API_KEY:
    import sys
    sys.path.insert(0, os.path.abspath("agentreview"))
    
    try:
        from agentreview.environments import PaperReview
        from agentreview.paper_review_arena import PaperReviewArena
        from agentreview.paper_review_settings import get_experiment_settings
        from agentreview.utility.experiment_utils import initialize_players
        from agentreview.utility.utils import get_paper_decision_mapping
        
        print("✅ AgentReview 모듈 로딩 완료")
        READY = True
    except Exception as e:
        print(f"❌ AgentReview 모듈 로딩 실패")
        print(f"   오류: {e}")
        print(f"\n해결 방법:")
        print(f"   - Cell 2를 먼저 실행하여 AgentReview 클론")
        READY = False
elif not USE_AGENTREVIEW:
    print("⚠️ 샘플 데이터가 없어 실행할 수 없습니다.")
    print("   Cell 4를 확인하세요.")
    READY = False
elif not OPENAI_API_KEY:
    print("⚠️ OpenAI API Key가 없어 실행할 수 없습니다.")
    print("   .env 파일에 OPENAI_API_KEY를 추가하세요.")
    READY = False

In [ ]:
# Cell 7: Demo 실행 (샘플 논문 2개)
if READY:
    from argparse import Namespace
    
    # 설정
    args = Namespace(
        openai_client_type='openai',
        conference='ICLR2022',  # 샘플 데이터 연도
        num_reviewers_per_paper=3,
        model_name='gpt-4o-mini',  # 비용 절감
        output_dir='outputs',
        data_dir='./data',
        acceptance_rate=0.32,
        skip_logging=False,
        task='paper_review'
    )
    
    # 논문 ID 매핑
    paper_id2decision, paper_decision2ids = get_paper_decision_mapping(
        args.data_dir, 
        args.conference
    )
    
    # 샘플 논문 2개 (Accept 1개 + Reject 1개)
    sample_ids = [284, 216]  # Accept, Reject
    
    # 🔧 여기서 리뷰어 설정 변경 가능!
    # 옵션: baseline_setting, malicious_setting, benign_setting, 
    #       expert_mix_setting, worst_case_setting, fame_bias_setting
    selected_setting = baseline_setting  # <- 이 부분을 바꿔보세요!
    
    print(f"📝 {len(sample_ids)}개 논문 리뷰 시작...")
    print(f"🎭 리뷰어 설정: {selected_setting['reviewer']}\n")
    
    for paper_id in sample_ids:
        paper_decision = paper_id2decision[str(paper_id)]
        
        experiment_setting = get_experiment_settings(
            paper_id=paper_id,
            paper_decision=paper_decision,
            setting=selected_setting
        )
        
        print(f"\n{'='*60}")
        print(f"📄 Paper {paper_id}")
        print(f"   실제 결과: {paper_decision}")
        print(f"   리뷰어: {experiment_setting['reviewer']}")
        print(f"{'='*60}\n")
        
        # 플레이어 초기화
        players = initialize_players(
            experiment_setting=experiment_setting, 
            args=args
        )
        
        # 환경 생성
        env = PaperReview(
            player_names=[p.name for p in players],
            paper_decision=paper_decision,
            paper_id=paper_id,
            args=args,
            experiment_setting=experiment_setting
        )
        
        # Arena 실행
        arena = PaperReviewArena(
            players=players, 
            environment=env, 
            args=args
        )
        
        # 5단계 파이프라인 실행
        arena.launch_cli(interactive=False)
        
        print("\n✅ 리뷰 완료\n")
else:
    print("⚠️ 위 셀들을 먼저 실행하세요 (데이터 또는 API key 확인 필요)")

---
## 결과 확인

In [ ]:
# Cell 8: 생성된 파일 확인
if READY and os.path.exists('outputs'):
    print("📊 생성된 리뷰 파일:")
    
    files = sorted(os.listdir('outputs'))
    print(f"\n📁 총 {len(files)}개 파일 생성\n")
    
    # 최근 파일 10개
    print("최근 생성된 파일:")
    for f in files[-10:]:
        print(f"  - {f}")
    
    print("\n💡 Tip: outputs/ 폴더에서 각 단계별 리뷰 내용을 확인할 수 있습니다.")
elif READY:
    print("⚠️ outputs/ 폴더가 없습니다.")
    print("   Cell 7을 먼저 실행하여 리뷰를 생성하세요.")
else:
    print("⚠️ AgentReview가 준비되지 않았습니다.")

---
## DIY: 설정 비교 실험

In [ ]:
# DIY Cell 9: 나만의 리뷰어 조합 만들기

# 조합 가능한 속성:
# - Intention: benign (호의적), malicious (악의적), BASELINE (중립)
# - Knowledge: knowledgeable (전문가), unknowledgeable (비전문가), BASELINE (보통)
# - Commitment: responsible (성실), irresponsible (불성실), BASELINE (보통)
# - Fame: authors_are_famous, authors_are_unfamous
#
# 언더스코어(_)로 연결하여 조합:
# 예: "knowledgeable_benign", "malicious_unknowledgeable_irresponsible"

# 실험 아이디어 1: 전문가들만
expert_only_setting = {
    "reviewer": ["knowledgeable_responsible", "knowledgeable_benign", "knowledgeable"],
    "AC": ["BASELINE"],
    "author": ["BASELINE"],
    "global_settings": {
        "provides_numeric_rating": ['reviewer', 'ac'],
        "persons_aware_of_authors_identities": []
    }
}

# 실험 아이디어 2: 다양한 성향 혼합
diverse_setting = {
    "reviewer": ["knowledgeable_malicious", "benign_irresponsible", "unknowledgeable_responsible"],
    "AC": ["BASELINE"],
    "author": ["BASELINE"],
    "global_settings": {
        "provides_numeric_rating": ['reviewer', 'ac'],
        "persons_aware_of_authors_identities": []
    }
}

# 실험 아이디어 3: 당신만의 조합 (아래 수정!)
my_custom_setting = {
    "reviewer": ["BASELINE", "BASELINE", "BASELINE"],  # <- 여기를 바꿔보세요!
    "AC": ["BASELINE"],
    "author": ["BASELINE"],
    "global_settings": {
        "provides_numeric_rating": ['reviewer', 'ac'],
        "persons_aware_of_authors_identities": []
    }
}

print("💡 커스텀 설정 예시:")
print(f"  전문가만: {expert_only_setting['reviewer']}")
print(f"  다양한 혼합: {diverse_setting['reviewer']}")
print(f"  나만의 조합: {my_custom_setting['reviewer']}")
print("\n🔧 Cell 7에서 selected_setting을 my_custom_setting으로 바꿔서 실행해보세요!")

---
## 토론

### 질문

1. **리뷰어 페르소나 조합의 영향**
   - `malicious` vs `malicious_unknowledgeable` - 차이가 있나요?
   - 3명 중 1명이 `knowledgeable_responsible`이면 품질이 올라가나요?
   - `authors_are_famous` vs `authors_are_unfamous` - 유명도 편향이 얼마나 큰가요?

2. **5단계 파이프라인의 가치**
   - Author rebuttal이 점수를 올리나요?
   - AC 토론이 편향을 줄이나요?
   - `irresponsible` 리뷰어도 rebuttal 후 점수를 바꾸나요?

3. **실제 활용**
   - Peer review 시스템 개선에 어떻게 사용?
   - 논문 투고 전 사전 테스트? (어떤 조합으로?)
   - 리뷰어 배정 최적화? (최소 몇 명이 `knowledgeable`이어야 할까?)

4. **실험 아이디어**
   - `worst_case_setting` vs `expert_only_setting` 비교
   - 같은 논문에 다양한 설정 적용하여 결과 비교
   - 어떤 조합이 실제 ICLR 리뷰와 가장 유사할까?